In [1]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import parallel_bulk

import pandas as pd
import numpy as np
import re

In [2]:
es = Elasticsearch([{'host':'localhost', 'port':9200}])

In [3]:
es.ping()

True

In [4]:
corpus = open('ARC_Corpus.txt', 'r', encoding='utf-8')

In [5]:
corpus_1 = open('Aristo-Mini-Corpus-Dec2016.txt', 'r', encoding='utf-8')

In [6]:
b = {"mappings":{
        "properties":{
            "sentence":{
                "type":"text"
            }
        }
}
    }

In [7]:
ret = es.indices.create(index='corpus', ignore=400, body=b) #ARC Corpus.
ret_2 = es.indices.create(index='corpus_1', ignore=400, body=b) #Aristo Mini Corpus.

In [8]:
def gen_data_arc():
    '''
    This function will yield a sentence from the ARC corpus and the index name.
    '''
    with open('ARC_Corpus.txt', 'r', encoding='utf-8') as viola:
        for row in viola: 
            yield{
                "_index":"corpus",
                 "sentence":row
                 }          

In [9]:
parallel_bulk(es, gen_data_arc())

<generator object parallel_bulk at 0x0000029404028048>

In [10]:
def gen_data_aristo():
    '''
    This function will yield a sentence from the ARC corpus and the index name.
    '''
    with open('Aristo-Mini-Corpus-Dec2016.txt', 'r', encoding='utf-8') as viola:
        for row in viola: 
            yield{
                "_index":"corpus_1",
                 "sentence":row
                 } 

In [11]:
parallel_bulk(es, gen_data_aristo())

<generator object parallel_bulk at 0x0000029404028F48>

In [12]:
df_easy_train = pd.read_csv('df_easy_train')
df_easy_dev = pd.read_csv('df_easy_dev')
df_easy_test = pd.read_csv('df_easy_test')

In [13]:
df_challenge_train = pd.read_csv('df_challenge_train.csv')
df_challenge_dev = pd.read_csv('df_challenge_dev.csv')
df_challenge_test = pd.read_csv('df_challenge_test.csv')

In [14]:
df_easy_train.head()

,Unnamed: 0,AnswerKey,question,only_question,quesiton_length,only_answers,context
0,0,B,Which factor will most likely cause a person t...,Which factor will most likely cause a person t...,66,(A) a leg muscle relaxing after exercise (B) a...,Piercings show that a person is rebelious and ...
1,1,B,Lichens are symbiotic organisms made of green ...,Lichens are symbiotic organisms made of green ...,139,(A) carbon dioxide (B) food (C) protection (D)...,Lichens are comprised of blue-green or green a...
2,2,D,When a switch is used in an electrical circuit...,When a switch is used in an electrical circuit...,63,(A) cause the charge to build. (B) increase an...,Rotary switch A ` rotary switch ' is an electr...
3,3,A,Which of the following is an example of an ass...,Which of the following is an example of an ass...,61,(A) contact lens (B) motorcycle (C) raincoat (...,Which of the following is an example of an ang...
4,4,3,"Rocks are classified as igneous, metamorphic, ...","Rocks are classified as igneous, metamorphic, ...",74,(1) their color (2) their shape (3) how they f...,"In one simplistic and linear example, elements..."


In [15]:
def data_generator(a):
    '''
    This function will accept a data point and returns a list of options
    converts the only_options list of options.

    Output will be as follows:

    options = [option_1, option_2, option_3, option_4, option_5]                   

    Note: If there's no option E then it will be written as 'None of the above'.
    '''
    options = []

    if '(A)' in a:
        option_1 = a.split('(B)')[0].replace('(A)','').lstrip()

        b = ' '.join(i for i in a.split() if i not in option_1)
        option_2 = b.split('(C)')[0].replace('(B)','').replace('(A)','').lstrip()

        c = ' '.join(i for i in b.split() if i not in option_2)
        option_3 = c.split('(D)')[0].replace('(C)','').replace('(B)','').replace('(A)','').lstrip()

        if '(D)' not in c:
            option_4 = 'None of the above'
            option_5 = 'None of the above'
        else:
            if '(E)' not in c:
                option_4 = c.split('(D)')[1].lstrip()
                option_5 = 'None of the above'
            else:
                d = ' '.join(i for i in c.split() if i not in option_3)
                option_4 = d.split('(E)')[0].replace('(D)','').replace('(C)','').replace('(B)','').replace('(A)','').lstrip()
                option_5 = d.split('(E)')[1].lstrip()
    else:
        option_1 = a.split('(2)')[0].replace('(1)','').lstrip()

        b = ' '.join(i for i in a.split() if i not in option_1)
        option_2 = b.split('(3)')[0].replace('(2)','').replace('(1)','').lstrip()

        c = ' '.join(i for i in b.split() if i not in option_2)
        option_3 = c.split('(4)')[0].replace('(3)','').replace('(2)','').replace('(1)','').lstrip()

        if '(4)' in c:
            option_4 = c.split('(4)')[1].lstrip()
        else:
            option_4 = 'None of the above'
        option_5 = 'None of the above'

    options = [option_1, option_2, option_3, option_4, option_5]

    return options

In [16]:
df_easy_dev['options_list'] = df_easy_dev['only_answers'].map(data_generator)
df_easy_train['options_list'] = df_easy_train['only_answers'].map(data_generator)
df_easy_test['options_list'] = df_easy_test['only_answers'].map(data_generator)

In [17]:
df_easy_train.head()

,Unnamed: 0,AnswerKey,question,only_question,quesiton_length,only_answers,context,options_list
0,0,B,Which factor will most likely cause a person t...,Which factor will most likely cause a person t...,66,(A) a leg muscle relaxing after exercise (B) a...,Piercings show that a person is rebelious and ...,"[a leg muscle relaxing after exercise , bacter..."
1,1,B,Lichens are symbiotic organisms made of green ...,Lichens are symbiotic organisms made of green ...,139,(A) carbon dioxide (B) food (C) protection (D)...,Lichens are comprised of blue-green or green a...,"[carbon dioxide , food , protection , water, N..."
2,2,D,When a switch is used in an electrical circuit...,When a switch is used in an electrical circuit...,63,(A) cause the charge to build. (B) increase an...,Rotary switch A ` rotary switch ' is an electr...,"[cause the charge to build. , increase and dec..."
3,3,A,Which of the following is an example of an ass...,Which of the following is an example of an ass...,61,(A) contact lens (B) motorcycle (C) raincoat (...,Which of the following is an example of an ang...,"[contact lens , motorcycle , raincoat , coffee..."
4,4,3,"Rocks are classified as igneous, metamorphic, ...","Rocks are classified as igneous, metamorphic, ...",74,(1) their color (2) their shape (3) how they f...,"In one simplistic and linear example, elements...","[their color , shape , how they formed , miner..."


In [12]:
def get_context(question, options):
    '''
    This function will return a context after joining a question
    and its options separately. 
    For example: question + option_1
                 question + option_2
                 question + option_3
                 question + option_4
   '''
    scores = []
    sentences = []
    choices = []
    
    #Dividing options into separate parts.
    if '(A)' in options:

        split_1 = options.split('(B)')
        option_1 = split_1[0].replace('(A)','')
        choices.append(option_1)

        split_2 = split_1[1].split('(C)')
        option_2 = split_2[0]
        choices.append(option_2)

        if '(D)' not in split_2[1]:
            option_3 = split_2[1]
            choices.append(option_3)

        elif '(D)' in split_2[1]:
            split_3 = split_2[1].split('(D)')
            option_3 = split_3[0]
            choices.append(option_3)

            if '(E)' not in split_3[1]:
                option_4 = split_3[1]
                choices.append(option_4)

            else:
                split_4 = split_3[1].split('(E)')
                option_4 = split_4[0]
                choices.append(option_4)

                option_5 = split_4[1]
                choices.append(option_5)
    else:
        split_1 = options.split('(2)')
        option_1 = split_1[0].replace('(1)','')
        choices.append(option_1)

        split_2 = split_1[1].split('(3)')
        option_2 = split_2[0]
        choices.append(option_2)

        if '(4)' not in split_2[1]:
            option_3 = split_2[1]
            choices.append(option_3)

        elif '(4)' in split_2[1]:
            split_3 = split_2[1].split('(4)')
            option_3 = split_3[0]
            choices.append(option_3)

            if '(5)' not in split_3[1]:
                option_4 = split_3[1]
                choices.append(option_4)

            else:
                split_4 = split_3[1].split('(5)')
                option_4 = split_4[0]
                choices.append(option_4)

                option_5 = split_4[1]
                choices.append(option_5)
    
    for i in [choices]:
        #Defining query to be searched for.
        search_param = {"size": 100,
                        "query": {
                            "bool": {
                                "must": [
                                    {"match": {
                                        "sentence": question + f'{i}'
                                    }}
                                ],
                                "filter": [
                                    {"match": {"sentence": f'{i}'}}
                                ]
                            }
                        }}

        #Searching in the 'corpus' index.
        a = es.search(index='corpus', body=search_param)

        for i in a['hits']['hits']:
            sentence = i['_source']['sentence']
            if sentence in sentences:
                continue
            else:
                sentences.append(sentence)
                score = i['_score']
                scores.append(score)
                
    data = pd.DataFrame()
    data['score'] = scores
    data['sentence'] = sentences
    
    #Sorting by score to select top K sentences.
    data.sort_values(by='score', ascending=False, inplace=True)
    
    context = ' '.join(data['sentence'].values[:20]) #Selecting top 20 sentences for the context.
    
    return context

In [58]:
def get_context_for_each_candidate(question, options_list):
    '''
    This function will return a context after joining a question
    and its options separately. 
    For example: question + option_1
                 question + option_2
                 question + option_3
                 question + option_4
   '''
    option_1_docs = dict()
    option_2_docs = dict()
    option_3_docs = dict()
    option_4_docs = dict()
    option_5_docs = dict()
    choices = options_list
    
    
    for j in range(len(choices)):
        
        #Defining query to be searched for.
        search_param = {"size": 50,
                        "query": {
                            "bool": {
                                "must": [
                                    {"match": {
                                        "sentence": question + ' ' + f'{choices[j]}'
                                    }}
                                ],
                                "filter": [
                                    {"match": {"sentence": f'{choices[j]}'}}
                                ]
                            }
                        }}

        #Searching in the 'corpus' and 'corpus_1' index.
        a = es.search(index='corpus', body=search_param)
        b = es.search(index='corpus_1', body=search_param)
        
        sentences = []
        for i in a['hits']['hits']:
            
            sent_tf_idf_score = dict()
            sentence = i['_source']['sentence']
            if sentence in sentences:
                continue
            else:
                sentences.append(sentence)
                score = i['_score']
                
                if j == 0:
                    option_1_docs[sentence] = score
                elif j == 1:
                    option_2_docs[sentence] = score
                elif j == 2:
                    option_3_docs[sentence] = score
                elif j == 3:
                    option_4_docs[sentence] = score
                elif j == 4:
                    option_5_docs[sentence] = score
        
        sentences = []
        for i in b['hits']['hits']:
            
            sent_tf_idf_score = dict()
            sentence = i['_source']['sentence']
            if sentence in sentences:
                continue
            else:
                sentences.append(sentence)
                score = i['_score']
                
                if j == 0:
                    option_1_docs[sentence] = score
                elif j == 1:
                    option_2_docs[sentence] = score
                elif j == 2:
                    option_3_docs[sentence] = score
                elif j == 3:
                    option_4_docs[sentence] = score
                elif j == 4:
                    option_5_docs[sentence] = score
                
    
    return pd.Series([option_1_docs, option_2_docs, option_3_docs, option_4_docs, option_5_docs])

In [45]:
a = df_easy_train['only_question'].iloc[1139]
b = df_easy_train['options_list'].iloc[1139]

In [48]:
b

['atom. ', 'neutron. ', 'molecule. ', 'electron.', 'None of the above']

In [46]:
q,w,e,r,t = get_context_for_each_candidate(a,b)

In [59]:
df_easy_train[['option_1_docs','option_2_docs','option_3_docs','option_4_docs','option_5_docs',]] = df_easy_train[['only_question','options_list']].apply(lambda x: get_context_for_each_candidate(*x), axis=1)

In [60]:
df_easy_dev[['option_1_docs','option_2_docs','option_3_docs','option_4_docs','option_5_docs',]] = df_easy_dev[['only_question','options_list']].apply(lambda x: get_context_for_each_candidate(*x), axis=1)
df_easy_test[['option_1_docs','option_2_docs','option_3_docs','option_4_docs','option_5_docs',]] = df_easy_test[['only_question','options_list']].apply(lambda x: get_context_for_each_candidate(*x), axis=1)

In [61]:
df_easy_train.head()

,Unnamed: 0,AnswerKey,question,only_question,quesiton_length,only_answers,context,options_list,option_1_docs,option_2_docs,option_3_docs,option_4_docs,option_5_docs
0,0,B,Which factor will most likely cause a person t...,Which factor will most likely cause a person t...,66,(A) a leg muscle relaxing after exercise (B) a...,Piercings show that a person is rebelious and ...,"[a leg muscle relaxing after exercise , bacter...",{'-' Which joint of the leg con- tains most mu...,{'When the larvae of Trichinella worms get int...,{'Environmental perturbations are most likely ...,{'Carbohydrates will most likely be the larges...,{'Which technique was most likely used to deve...
1,1,B,Lichens are symbiotic organisms made of green ...,Lichens are symbiotic organisms made of green ...,139,(A) carbon dioxide (B) food (C) protection (D)...,Lichens are comprised of blue-green or green a...,"[carbon dioxide , food , protection , water, N...",{'The hydras let these algae live inside them ...,"{'algae near the surface of ponds, lakes, and ...","{'In this symbiotic relationship, the algae ga...",{'This is a small genus of marine and brackish...,{'Lichens are comprised of blue-green or green...
2,2,D,When a switch is used in an electrical circuit...,When a switch is used in an electrical circuit...,63,(A) cause the charge to build. (B) increase an...,Rotary switch A ` rotary switch ' is an electr...,"[cause the charge to build. , increase and dec...",{'If an ordinary switch is used to control the...,{'And when the electrical switch was turned OF...,{'If an ordinary switch is used to control the...,{'When the circuit switch is open (Figure 2) c...,{'If an ordinary switch is used to control the...
3,3,A,Which of the following is an example of an ass...,Which of the following is an example of an ass...,61,(A) contact lens (B) motorcycle (C) raincoat (...,Which of the following is an example of an ang...,"[contact lens , motorcycle , raincoat , coffee...",{'An instrument for measuring the degree of cu...,{'The characters are typically members of an o...,"{'[DUSSEL hurriedly takes off the raincoat, gi...",{'coffee n a beverage consisting of an infusio...,{'Which of the following is an example of an a...
4,4,3,"Rocks are classified as igneous, metamorphic, ...","Rocks are classified as igneous, metamorphic, ...",74,(1) their color (2) their shape (3) how they f...,"In one simplistic and linear example, elements...","[their color , shape , how they formed , miner...","{'In one simplistic and linear example, elemen...",{'Western carrot cultivars are commonly classi...,"{'- formed by “changing” preexisting igneous, ...",{'This group of minerals is of common occur- r...,"{'The second class of rocks, sedimentary, is ..."


In [62]:
df_easy_train.columns

Index(['Unnamed: 0', 'AnswerKey', 'question', 'only_question',
       'quesiton_length', 'only_answers', 'context', 'options_list',
       'option_1_docs', 'option_2_docs', 'option_3_docs', 'option_4_docs',
       'option_5_docs'],
      dtype='object')

In [63]:
df_easy_train.drop('Unnamed: 0', axis=1, inplace=True)
df_easy_dev.drop('Unnamed: 0', axis=1, inplace=True)
df_easy_test.drop('Unnamed: 0', axis=1, inplace=True)

In [64]:
df_easy_train.to_csv('df_easy_train_3.csv', index=False)
df_easy_dev.to_csv('df_easy_dev_3.csv', index=False)
df_easy_test.to_csv('df_easy_test_3.csv', index=False)

In [13]:
list_of_context = []

for i in range(len(df_easy_train)):
    question = df_easy_train['only_question'].iloc[i]
    answers = df_easy_train['only_answers'].iloc[i]
    
    context = get_context(question, answers)
    list_of_context.append(context)

In [15]:
df_easy_train['context_2'] = list_of_context

In [16]:
df_easy_train.head()

,Unnamed: 0,AnswerKey,question,only_question,quesiton_length,only_answers,context,context_2
0,0,B,Which factor will most likely cause a person t...,Which factor will most likely cause a person t...,66,(A) a leg muscle relaxing after exercise (B) a...,Piercings show that a person is rebelious and ...,"As a result, the food will not be digested pro..."
1,1,B,Lichens are symbiotic organisms made of green ...,Lichens are symbiotic organisms made of green ...,139,(A) carbon dioxide (B) food (C) protection (D)...,Lichens are comprised of blue-green or green a...,"In this symbiotic relationship, the algae gain..."
2,2,D,When a switch is used in an electrical circuit...,When a switch is used in an electrical circuit...,63,(A) cause the charge to build. (B) increase an...,Rotary switch A ` rotary switch ' is an electr...,If an ordinary switch is used to control the w...
3,3,A,Which of the following is an example of an ass...,Which of the following is an example of an ass...,61,(A) contact lens (B) motorcycle (C) raincoat (...,Which of the following is an example of an ang...,The characters are typically members of an out...
4,4,3,"Rocks are classified as igneous, metamorphic, ...","Rocks are classified as igneous, metamorphic, ...",74,(1) their color (2) their shape (3) how they f...,"In one simplistic and linear example, elements...","In one simplistic and linear example, elements..."


In [17]:
df_easy_dev['context_2'] = df_easy_dev[['only_question','only_answers']].apply(lambda x: get_context(*x), axis=1)

In [18]:
df_easy_dev.head()

,Unnamed: 0,AnswerKey,question,only_question,quesiton_length,only_answers,context,context_2
0,0,A,Which technology was developed most recently? ...,Which technology was developed most recently?,46,(A) cellular telephone (B) television (C) refr...,This technology has been developed by Stroma M...,With Voice over IP -LRB- VoIP -RRB- technology...
1,1,C,A student hypothesizes that algae are producer...,A student hypothesizes that algae are producer...,121,(A) Do algae consume other organisms? (B) Whic...,This workshop will help suppliers determine wh...,Algae: Algae are plant-like organisms that hav...
2,2,A,Soccer players use their muscle systems to kic...,Soccer players use their muscle systems to kic...,111,(A) The nervous system (B) The endocrine syste...,The increased use of a muscle with its various...,"- Cardiovascular system, G.I. system, Endocrin..."
3,3,B,Planets in the solar system are in constant mo...,Planets in the solar system are in constant mo...,118,(A) the size of the planets (B) gravitational ...,"Thus, to compute the orbits of planets in the ...","Within the Solar System , the terrestrial plan..."
4,4,B,How is a pond different from a lake? (A) Ponds...,How is a pond different from a lake?,37,(A) Ponds have moving water. (B) Ponds are sma...,A pond is still and shallow and much smaller t...,"However, a small percentage will disperse to d..."


In [19]:
df_easy_test['context_2'] = df_easy_test[['only_question','only_answers']].apply(lambda x: get_context(*x), axis=1)

In [20]:
df_easy_test.head()

,Unnamed: 0,AnswerKey,question,only_question,quesiton_length,only_answers,context,context_2
0,0,A,Which statement best explains why photosynthes...,Which statement best explains why photosynthes...,86,(A) Sunlight is the source of energy for nearl...,Which statement best explains why the solution...,Coral reefs are among the most productive and ...
1,1,B,Which piece of safety equipment is used to kee...,Which piece of safety equipment is used to kee...,98,(A) safety goggles (B) breathing mask (C) rubb...,Clean harvest equipment before entering the fi...,"For safety , players wear protective goggles a..."
2,2,D,Meiosis is a type of cell division in which ge...,Meiosis is a type of cell division in which ge...,114,(A) brain cells (B) bone cells (C) muscle cell...,Meiosis is a type of cell division in which a ...,Meiosis is a type of cell division in which a ...
3,3,D,Which characteristic describes the texture of ...,Which characteristic describes the texture of ...,62,(A) gray (B) warm (C) long (D) soft,Which of the following best describes the patt...,"It is of good weather resistant texture, which..."
4,4,B,Which best describes the structure of an atom?...,Which best describes the structure of an atom?,47,(A) a lightweight core surrounded by neutral p...,Which of the following best describes the patt...,Histones package and compact DNA by assembling...


In [26]:
df_challenge_train.head()

,Unnamed: 0,Unnamed: 0.1,AnswerKey,question,only_question,quesiton_length,context,only_answers
0,0,0,A,George wants to warm his hands quickly by rubb...,George wants to warm his hands quickly by rubb...,103,The uneven surface reflects heat into other ar...,(A) dry palms (B) wet palms (C) palms covered ...
1,1,1,B,Which of the following statements best explain...,Which of the following statements best explain...,98,Which of the following statements best explain...,(A) The refrigerator door is smooth. (B) The r...
2,2,2,B,A fold observed in layers of sedimentary rock ...,A fold observed in layers of sedimentary rock ...,76,A fold in rock layers that forms an arch. _ Th...,(A) cooling of flowing magma. (B) converging o...
3,3,3,D,Which of these do scientists offer as the most...,Which of these do scientists offer as the most...,141,"flowering plants, Mesozoic Era The Triassic pe...",(A) worldwide disease (B) global mountain buil...
4,4,4,B,A boat is acted on by a river current flowing ...,A boat is acted on by a river current flowing ...,198,The direction in which a skipper wishes to ste...,(A) west (B) east (C) north (D) south


In [27]:
df_challenge_train['context_2'] = df_challenge_train[['only_question','only_answers']].apply(lambda x: get_context(*x), axis=1)
df_challenge_dev['context_2'] = df_challenge_dev[['only_question','only_answers']].apply(lambda x: get_context(*x), axis=1)
df_challenge_test['context_2'] = df_challenge_test[['only_question','only_answers']].apply(lambda x: get_context(*x), axis=1)

In [37]:
q = df_easy_train['context_2'].iloc[0]
q

'As a result, the food will not be digested properly and you will get a heavy bloated feeling in the stomach.\n —Points or tracts on the skin of a hysterical person pressure on which will induce a hysterical attack.\n 2) After several generations, most of the organisms in the population will have that adaptation.\n As a result of the absence of violets, the fritillary population will MOST LIKELY:\n -\' Which joint of the leg con- tains most muscle.\'\n The local anesthetic is injected into the skin and down to the muscle, and after the area is numb a small incision is made in the skin and a passage made through the skin and muscle into the chest.\n Which process has the greatest effect in determining which members of a population are most likely to survive until reproductive age?\n On being threatened, a Binturong will defecate or urinate on the animal or person which is threatening it.\n When the larvae of Trichinella worms get into the bloodstream, they most likely will\n Carbohydrat

Here we can see that there's a lot of line breaks in context, i.e. \n.
We will simply replace them with empty string as they are not useful.

In [29]:
def replace_line_breaks(context):
    '''
    Function to replace '\n' with empty strings('').
    '''
    return context.replace('\n','')

In [38]:
df_easy_train['context_2'] = df_easy_train['context_2'].apply(replace_line_breaks)

In [39]:
df_easy_train['context_2'].iloc[0]

'As a result, the food will not be digested properly and you will get a heavy bloated feeling in the stomach. —Points or tracts on the skin of a hysterical person pressure on which will induce a hysterical attack. 2) After several generations, most of the organisms in the population will have that adaptation. As a result of the absence of violets, the fritillary population will MOST LIKELY: -\' Which joint of the leg con- tains most muscle.\' The local anesthetic is injected into the skin and down to the muscle, and after the area is numb a small incision is made in the skin and a passage made through the skin and muscle into the chest. Which process has the greatest effect in determining which members of a population are most likely to survive until reproductive age? On being threatened, a Binturong will defecate or urinate on the animal or person which is threatening it. When the larvae of Trichinella worms get into the bloodstream, they most likely will Carbohydrates will most likel

In [40]:
df_easy_dev['context_2'] = df_easy_dev['context_2'].apply(replace_line_breaks)
df_easy_test['context_2'] = df_easy_test['context_2'].apply(replace_line_breaks)

df_challenge_train['context_2'] = df_challenge_train['context_2'].apply(replace_line_breaks)
df_challenge_dev['context_2'] = df_challenge_dev['context_2'].apply(replace_line_breaks)
df_challenge_test['context_2'] = df_challenge_test['context_2'].apply(replace_line_breaks)

In [41]:
def preprocess_context(x):

    x = re.sub(r"\xa0", " ", x) #\xa0 represents space in Unicode. Thus removing it.

    #Decontracting the words.
    x = re.sub(r"won't", "will not", x)
    x = re.sub(r"can\'t", "can not", x)
    x = re.sub(r"n\'t", " not", x)
    x = re.sub(r"\'re", " are", x)
    x = re.sub(r"\'s", " is", x)
    x = re.sub(r"\'d", " would", x)
    x = re.sub(r"\'ll", " will", x)
    x = re.sub(r"\'t", " not", x)
    x = re.sub(r"\'ve", " have", x)
    x = re.sub(r"\'m", " am", x)
    x = re.sub(r"b/t", "between", x)
    x = re.sub(r"mph", "miles per hour", x)
    x = re.sub(r"km/h", "kilometers per hour", x)

    x = re.sub(r"[^A-Za-z0-9%+]+", " ", x) #Removing special characters.
    x = re.sub(r"\s+", " ", x) #Removing extra spaces.

    return x

In [42]:
df_easy_train['context'] = df_easy_train['context'].apply(preprocess_context)
df_easy_train['context'].iloc[0]

'Piercings show that a person is rebelious and likely to cause trouble Which technique was most likely used to develop this new variety of dog In which occupations might workers be at risk to develop Q fever For example patients with Nf2 are unfortunately more likely to develop meningiomas because they have inherited a gene which has the potential to cause normal cells to become cancerous When asked to draw pictures expressing different emotions which emotion would cause the most difficulty for a person with Urbach Wiethe disease Another cause which is a potential risk factor for formation of clots is a high homocysteine level Determining the most likely cause is a process of elimination eliminating potential causes of a problem It is most likely the person is not there His excess drinking was most likely the cause of a stroke that killed him in 1984 Which would most likely result if a short winged female mates with a'

In [43]:
df_easy_dev['context_2'] = df_easy_dev['context_2'].apply(preprocess_context)
df_easy_test['context_2'] = df_easy_test['context_2'].apply(preprocess_context)

df_challenge_train['context_2'] = df_challenge_train['context_2'].apply(preprocess_context)
df_challenge_dev['context_2'] = df_challenge_dev['context_2'].apply(preprocess_context)
df_challenge_test['context_2'] = df_challenge_test['context_2'].apply(preprocess_context)

In [46]:
max_len_context = 50 #Some random number.
min_len_context = 100 #Some random number.

for i in range(len(df_easy_train)):
    
    context_length = len(df_easy_train['context_2'].iloc[i].split())
    if context_length > max_len_context:
        max_len_context = context_length
    
    if context_length < min_len_context:
        min_len_context = context_length


print(f'Maximum length of context in easy train set is {max_len_context}.')
print(f'Minimum length of context in easy train set is {min_len_context}.')

Maximum length of context in easy train set is 724.
Minimum length of context in easy train set is 75.


In [45]:
max_len_context = 50 #Some random number.
min_len_context = 100 #Some random number.

for i in range(len(df_challenge_train)):
    
    context_length = len(df_challenge_train['context_2'].iloc[i].split())
    if context_length > max_len_context:
        max_len_context = context_length
    
    if context_length < min_len_context:
        min_len_context = context_length


print(f'Maximum length of context in challenge train set is {max_len_context}.')
print(f'Minimum length of context in challenge train set is {min_len_context}.')

Maximum length of context in challenge train set is 713.
Minimum length of context in challenge train set is 100.


Here we can see that the maximum context length is 724 and minimum is 44.

In [52]:
df_challenge_dev.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'AnswerKey', 'question', 'only_question',
       'quesiton_length', 'context', 'only_answers', 'context_2'],
      dtype='object')

In [50]:
df_easy_train.drop('Unnamed: 0', axis=1, inplace=True)
df_easy_dev.drop('Unnamed: 0', axis=1, inplace=True)
df_easy_test.drop('Unnamed: 0', axis=1, inplace=True)

In [53]:
df_challenge_train.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1, inplace=True)
df_challenge_dev.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1, inplace=True)
df_challenge_test.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1, inplace=True)

In [54]:
df_easy_train.to_csv('df_easy_train_2.csv', index=False)
df_easy_dev.to_csv('df_easy_dev_2.csv', index=False)
df_easy_test.to_csv('df_easy_test_2.csv', index=False)

df_challenge_train.to_csv('df_challenge_train_2.csv', index=False)
df_challenge_dev.to_csv('df_challenge_dev_2.csv', index=False)
df_challenge_test.to_csv('df_challenge_test_2.csv', index=False)